In [6]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import statsmodels.api as sm

In [3]:
# Crea una carpeta para guardar los gráficos si no existe
output_dir = Path("../output_graphs")
output_dir.mkdir(exist_ok=True)

# Crea una carpeta para guardar las estadísticas si no existe
stats_dir = Path("../stats")
stats_dir.mkdir(exist_ok=True, parents=True)

# Crea una carpeta para guardar los datos procesados si no existe
data_dir = Path("../data/2_processed")
data_dir.mkdir(exist_ok=True, parents=True)

# Procesamiento de datos

## Encuestas

In [3]:
df = pd.read_excel('../data/1_raw/data_upao.xlsx', sheet_name=None)
df_cuestionario = df['cuestionario_sin_procesar']

# rename the columns to lowercase and snake_case
df_cuestionario.columns = [col.lower().replace(" ", "_") for col in df_cuestionario.columns]

ls_metrics = [col for col in df_cuestionario.columns if ('unnamed' not in col.lower() and 'trabajador' not in col.lower())]
ls_items = ['frecuencia', 'intensidad', 'severidad']

ls_columns = []
for metric in ls_metrics:
    for item in ls_items:
        ls_columns.append(f"{metric}_{item}")

df_cuestionario.columns = ls_columns

# elimna la primera fila
df_cuestionario = df_cuestionario.iloc[1:, :].reset_index(drop=True)

# asegura que todos los datos sean integers by converting each column to numeric (coercing errors to NaN)
df_cuestionario = df_cuestionario.apply(pd.to_numeric, errors='coerce').astype('Int64')

In [4]:
# crea una funcion para verificar que la severidad sea la multiplicacion de frecuencia e intensidad
def verify_severity(row):
    for metric in ls_metrics:
        frecuencia = row[f"{metric}_frecuencia"]
        intensidad = row[f"{metric}_intensidad"]
        severidad = row[f"{metric}_severidad"]
        if pd.isna(frecuencia) or pd.isna(intensidad) or pd.isna(severidad):
            continue
        if severidad != frecuencia * intensidad:
            return False
    return True

df_cuestionario['severity_check'] = df_cuestionario.apply(verify_severity, axis=1)
df_cuestionario['severity_check'].sum() == len(df_cuestionario)  # should be True

np.True_

In [5]:
# calcula la columna 'sindrome_visual_informatico'
df_cuestionario['puntaje_sindrome_visual_informatico'] = df_cuestionario[[f"{metric}_severidad" for metric in ls_metrics]].sum(axis=1)
df_cuestionario['svi'] = np.where(df_cuestionario['puntaje_sindrome_visual_informatico']>=6, 1, 0)

In [6]:
df_encuesta = df['ENCUESTA']
df_encuesta.columns = [col.lower().replace(" ", "_") for col in df_encuesta.columns]

df_final = pd.concat([df_cuestionario, df_encuesta], axis=1)
df_final[df_final.select_dtypes(include=['object']).columns] = (
    df_final[df_final.select_dtypes(include=['object']).columns]
    .apply(lambda col: col.str.replace(r',\d+', '', regex=True))
    .apply(pd.to_numeric, errors='coerce')
    .astype('Int64')
)

## Recategorización de variables

In [7]:
with open("../data/1_raw/recategorizacion.json", "r", encoding="utf-8") as f:
    dict_recategorizacion = json.load(f)

# convierte las key de dict_recategorizacion de segundo nivel a integer
for key in dict_recategorizacion.keys():
    dict_recategorizacion[key] = {int(k): v for k, v in dict_recategorizacion[key].items()}


In [8]:
df_final['svi'] = df_final['svi'].map(dict_recategorizacion['svi']).astype("category")

df_final['sexo'] = df_final['sexo'].replace({2: 0})
df_final['sexo'] = df_final['sexo'].map(dict_recategorizacion['sexo']).astype("category")

df_final['estado_civil'] = df_final['estado_civil'].replace({1: 0, 2: 1, 3: 1, 4: 1})
df_final['estado_civil'] = df_final['estado_civil'].map(dict_recategorizacion['estado_civil']).astype("category")

df_final['ingresos_mensuales'] = df_final['ingresos_mensuales'].replace({1: 0, 2: 0, 3: 0, 4: 1})
df_final['ingresos_mensuales'] = df_final['ingresos_mensuales'].map(dict_recategorizacion['ingresos_mensuales']).astype("category")

df_final['condiciones_oculares'] = df_final['condiciones_oculares'].replace({1: 1, 2: 0})
df_final['condiciones_oculares'] = df_final['condiciones_oculares'].map(dict_recategorizacion['condiciones_oculares']).astype("category")

df_final['lentes'] = df_final['lentes'].replace({1: 1, 2: 0})
df_final['lentes'] = df_final['lentes'].map(dict_recategorizacion['lentes']).astype("category")

df_final['iluminacion'] = df_final['iluminacion'].replace({1: 1, 2: 0})
df_final['iluminacion'] = df_final['iluminacion'].map(dict_recategorizacion['iluminacion']).astype("category")

df_final['frecuencia_de_pausas'] = df_final['frecuencia_de_pausas'].replace({1: 1, 2: 1, 3: 0})
df_final['frecuencia_de_pausas'] = df_final['frecuencia_de_pausas'].map(dict_recategorizacion['frecuencia_de_pausas']).astype("category")

df_final['uso_de_dispositivos'] = df_final['uso_de_dispositivos'].replace({1: 0, 2: 0, 3: 1, 4: 1})
df_final['uso_de_dispositivos'] = df_final['uso_de_dispositivos'].map(dict_recategorizacion['uso_de_dispositivos']).astype("category")

df_final['distancia_hacia_el_monitor'] = df_final['distancia_hacia_el_monitor'].replace({1: 0, 2: 1, 3: 1})
df_final['distancia_hacia_el_monitor'] = df_final['distancia_hacia_el_monitor'].map(dict_recategorizacion['distancia_hacia_el_monitor']).astype("category")

df_final['tiempo_de_exposicion'] = np.where(df_final['tiempo_de_exposicion']<=12, 0, 1)
df_final['tiempo_de_exposicion'] = df_final['tiempo_de_exposicion'].map(dict_recategorizacion['tiempo_de_exposicion']).astype("category")

df_final['edad'] = np.where(df_final['edad']<=35, 0, 1)
df_final['edad'] = df_final['edad'].map(dict_recategorizacion['edad']).astype("category")

df_final['experiencia_radiologia'] = np.where(df_final['experiencia_radiologia']<=3, 0, 1)
df_final['experiencia_radiologia'] = df_final['experiencia_radiologia'].map(dict_recategorizacion['experiencia_radiologia']).astype("category")

df_final['duracion_de_jornada'] = np.where(df_final['duracion_de_jornada']<=10, 0, 1)
df_final['duracion_de_jornada'] = df_final['duracion_de_jornada'].map(dict_recategorizacion['duracion_de_jornada']).astype("category")


In [9]:
df_final.to_csv("../data/2_processed/df_final_labeled.csv", index=False)

## Diccionario de datos

In [10]:
# import openpyxl

# # Cargar el archivo
# file_path = "../data/1_raw/data_upao.xlsx"
# workbook = openpyxl.load_workbook(file_path)

# # Revisar si existe la hoja "ENCUESTA"
# if "ENCUESTA" in workbook.sheetnames:
#     sheet = workbook["ENCUESTA"]
#     comments = []
#     for row in sheet.iter_rows():
#         for cell in row:
#             if cell.comment:
#                 comments.append({
#                     "celda": cell.coordinate,
#                     "comentario": cell.comment.text
#                 })
# else:
#     comments = None

# # Extraer los encabezados y sus comentarios, luego armar un diccionario limpio
# sheet = workbook["ENCUESTA"]

# # Función para pasar encabezado a camel_case
# def to_camel_case(s: str) -> str:
#     s = s.strip().lower().replace(" ", "_")
#     return s

# # Diccionario final
# categorias_encuesta = {}

# for item in comments:
#     celda = item["celda"]
#     comentario = item["comentario"]
    
#     # Encabezado en esa celda
#     encabezado = sheet[celda].value
#     if encabezado is None:
#         continue
    
#     key = to_camel_case(encabezado)
    
#     # Parsear categorías del comentario (después del "JORGE:")
#     categorias = {}
#     for linea in comentario.split("\n"):
#         if ":" in linea and linea.strip()[0].isdigit():
#             num, texto = linea.split(":", 1)
#             categorias[int(num.strip())] = texto.strip().lower()
    
#     if categorias:
#         categorias_encuesta[key] = categorias

# with open(data_dir / 'categorias_encuesta.json', 'w', encoding='utf-8') as f:
#     json.dump(categorias_encuesta, f, indent=4, ensure_ascii=False)
# categorias_encuesta

In [11]:
# def to_camel_case(s: str) -> str:
#     parts = s.strip().split()
#     return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:]) if parts else s

# def parse_mapping(mapping_str: str) -> dict:
#     # Busca pares del tipo 'texto=número' (los delimitadores pueden ser espacios o comas)
#     pairs = re.findall(r'([^=,\n]+)\s*=\s*([<>]?\s*\d+(?:-\d+)?|\d+)', mapping_str)
#     if not pairs:
#         return {}
#     mapping = {}
#     for text, num in pairs:
#         try:
#             # Extrae el dígito (en caso de rangos o símbolos se limpia la cadena)
#             num_clean = re.sub(r'[^\d]', '', num)
#             key_int = int(num_clean)
#         except Exception:
#             continue
#         mapping[key_int] = text.strip().lower()
#     return mapping

# categorias_metricas = {
#     "fatiga_visual": parse_mapping("Si=1 No=0"),
#     "tiempo_de_exposicion_a_pantallas": parse_mapping("Horas"),
#     "edad": parse_mapping("Años"),
#     "sexo": parse_mapping("Masculino=0 Femenino=1"),
#     "estado_civil": parse_mapping("Casado=2 Soltero=1 Divorciado=3 o Viudo=4"),
#     "anios_de_experiencia_profesional": parse_mapping(""),
#     "ingresos_mensuales": parse_mapping("< 3000=0 3001-5000=1 5001-7000=2 >70000=3"),
#     "uso_de_correccion_visual": parse_mapping("Si=1 No=0"),
#     "condiciones_oculares_preexistentes": parse_mapping("Si=0 No=1"),
#     "pausas_durante_el_trabajo": parse_mapping("Frecuente=0, Ocasional=1, Nulo=2"),
#     "iluminacion_del_entorno_de_trabajo": parse_mapping("adecuada=0, inadecuada=1"),
#     "duracion_de_la_jornada_laboral": parse_mapping(""),
#     "distancia_promedio_al_monitor_mientras_trabaja": parse_mapping("< 30 cm =0 30-60 cm =1 > 60 cm =2"),
#     "uso_de_dispositivos_moviles_fuera_del_trabajo": parse_mapping("Si=1 No=0"),
# }

# with open(data_dir / 'data_categorias_metricas.json', 'w', encoding='utf-8') as f:
#     json.dump(categorias_metricas, f, indent=4, ensure_ascii=False)

In [12]:
# # une con categorias_encuesta en un solo diccionario con el nombre todas_categorias
# todas_categorias = {**categorias_metricas, **categorias_encuesta}
# # exportar a json
# with open(data_dir / 'todas_categorias.json', 'w', encoding='utf-8') as f:
#     json.dump(todas_categorias, f, indent=4, ensure_ascii=False)

# Análisis descriptivo

In [13]:
df_final.describe(include='all').T.to_csv(stats_dir / 'descriptive_statistics.csv')

In [14]:
# crea una paleta de colores personalizada con el nombre medical_palette que tenga sea sobrio y para una audiencia médica
medical_palette = sns.color_palette("Set2")
sns.set_palette(medical_palette)

## Univariado

In [15]:
# Crear carpeta para gráficos univariados
univariate_output_dir = output_dir / "univariate"
univariate_output_dir.mkdir(parents=True, exist_ok=True)

In [16]:
MAX_CATEGORIES = 50
cat_cols = [
    c for c in df_final.columns
    if (
        df_final[c].dtype == 'object'
        or df_final[c].dtype.name == 'category'
        or df_final[c].nunique(dropna=False) < 10
    )
    and df_final[c].nunique(dropna=False) <= MAX_CATEGORIES
]

In [17]:
for col in cat_cols:
    # Convertir la columna a string, aprovechando que ya es de tipo category o int
    ser = df_final[col].astype(str).fillna('NA')

    counts = ser.value_counts(dropna=False).sort_values(ascending=False)
    df_plot = counts.reset_index()
    df_plot.columns = ['category', 'count']

    plt.figure(figsize=(6,4))
    pal = medical_palette[:len(df_plot)]  # paleta con la longitud exacta
    ax = sns.barplot(data=df_plot, x='category', y='count', hue='category', palette=pal, dodge=False)
    # eliminar la leyenda si existe
    if ax.get_legend() is not None:
        ax.get_legend().remove()

    plt.title(f'Frecuencias de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.savefig(str(univariate_output_dir / f"bar_{col}.png"), bbox_inches='tight')
    plt.close()


In [18]:
# Exportar tabla de estadísticas descriptivas para variables categóricas
desc_cat = df_final[cat_cols].describe(include='all').T
desc_cat.to_csv(stats_dir / 'univariate_categorical_statistics.csv')

## Análisis bivariado

In [19]:
# Gráficos bivariados usando 'svi' como variable dependiente (target)
target = 'svi'
num_hue = df_final[target].nunique()
palette_used = medical_palette[:num_hue]

# Crear carpeta para gráficos bivariados
bivariate_output_dir = output_dir / "bivariate"
bivariate_output_dir.mkdir(parents=True, exist_ok=True)

# Seleccionar las columnas categóricas (excluyendo el target) ya definidas como 'category' o 'object'
cat_cols_bi = [
    c for c in df_final.columns 
    if c != target and (isinstance(df_final[c].dtype, pd.CategoricalDtype) or df_final[c].dtype == 'object')
]

for col in cat_cols_bi:
    # Gráfico de barras (countplot) usando catplot
    g_bar = sns.catplot(
        data=df_final, x=col, hue=target, kind='count', 
        palette=palette_used, height=4, aspect=1.5
    )
    g_bar.fig.suptitle(f'Distribución de SVI por {col} (Bar)')
    g_bar.set_axis_labels(col, 'Frecuencia')
    # Ajustar la leyenda a la derecha
    g_bar.fig.subplots_adjust(right=0.8)
    if g_bar._legend is not None:
        g_bar._legend.set_bbox_to_anchor((1.05, 0.5))
    g_bar.fig.tight_layout()
    g_bar.savefig(str(bivariate_output_dir / f"bivariate_{col}_bar.png"))
    plt.close(g_bar.fig)
    
    # Si target es de tipo category o boolean, mapeamos a códigos numéricos.
    if isinstance(df_final[target].dtype, pd.CategoricalDtype):
        target_num = df_final[target].cat.codes
    elif df_final[target].dtype == 'bool':
        target_num = df_final[target].astype(int)
    else:
        try:
            target_num = pd.to_numeric(df_final[target])
        except Exception:
            continue

    # Crear un DataFrame temporal para incluir la versión numérica del target
    df_plot_temp = df_final.copy()
    df_plot_temp['_target_num'] = target_num

    # Gráfico violin: asignar hue=col y legend=False para evitar el warning
    g_violin = sns.catplot(
        data=df_plot_temp, x=col, y='_target_num', kind='violin',
        hue=col, palette=palette_used, legend=False, height=4, aspect=1.5
    )
    g_violin.fig.suptitle(f'Distribución de SVI numérico por {col} (Violin)')
    g_violin.set_axis_labels(col, target)
    g_violin.fig.tight_layout()
    g_violin.savefig(str(bivariate_output_dir / f"bivariate_{col}_violin.png"))
    plt.close(g_violin.fig)

    # Gráfico de puntos: asignar hue=col y legend=False para evitar el warning
    g_point = sns.catplot(
        data=df_plot_temp, x=col, y='_target_num', kind='point',
        hue=col, palette=palette_used, legend=False, height=4, aspect=1.5
    )
    g_point.fig.suptitle(f'Media de SVI por {col} (Point)')
    g_point.set_axis_labels(col, target)
    g_point.fig.tight_layout()
    g_point.savefig(str(bivariate_output_dir / f"bivariate_{col}_point.png"))
    plt.close(g_point.fig)
    
    # Eliminar la columna temporal
    df_plot_temp.drop(columns=['_target_num'], inplace=True)


# Análisis inferencial

## Alpha de Cronbach

Mide la consistencia interna de un conjunto de ítems o preguntas que buscan medir la misma característica o constructo:


- $\alpha < 0.6 \quad \rightarrow \quad \text{baja consistencia (poco confiable).}$

- $0.6 \leq \alpha < 0.7 \quad \rightarrow \quad \text{aceptable.}$

- $0.7 \leq \alpha < 0.8 \quad \rightarrow \quad \text{buena.}$

- $0.8 \leq \alpha \leq 0.9 \quad \rightarrow \quad \text{muy buena.}$

- $\alpha > 0.9 \quad \rightarrow \quad \text{excelente, pero puede indicar redundancia (ítems demasiado parecidos).}$

In [20]:
# alpha de cronbach para las métricas de fatiga visual
from scipy.stats import pearsonr
from itertools import combinations

def cronbach_alpha(df_items):
    # Número de ítems
    k = df_items.shape[1]
    
    # Varianza de cada ítem
    item_variances = df_items.var(axis=0, ddof=1)
    
    # Varianza total del test (suma de los ítems)
    total_variance = df_items.sum(axis=1).var(ddof=1)
    
    # Cálculo del alpha de Cronbach
    if total_variance == 0:
        return np.nan  # Evitar división por cero
    alpha = (k / (k - 1)) * (1 - item_variances.sum() / total_variance)
    return alpha

In [21]:
def categorize_alpha(alpha):
    if np.isnan(alpha):
        return "no definido"
    if alpha < 0.6:
        return "baja consistencia"
    elif alpha < 0.7:
        return "aceptable"
    elif alpha < 0.8:
        return "buena"
    elif alpha <= 0.9:
        return "muy buena"
    else:
        return "excelente, pero puede indicar redundancia"

# Compute Cronbach's alpha for frequency, intensity and severity items
df_fatiga_freq = df_final[[f"{metric}_frecuencia" for metric in ls_metrics]]
alpha_freq = cronbach_alpha(df_fatiga_freq)

df_fatiga_int = df_final[[f"{metric}_intensidad" for metric in ls_metrics]]
alpha_int = cronbach_alpha(df_fatiga_int)

df_fatiga_sev = df_final[[f"{metric}_severidad" for metric in ls_metrics]]
alpha_sev = cronbach_alpha(df_fatiga_sev)

print("Cronbach's alpha para frecuencia: {:.3f} ({})".format(alpha_freq, categorize_alpha(alpha_freq)))
print("Cronbach's alpha para intensidad: {:.3f} ({})".format(alpha_int, categorize_alpha(alpha_int)))
print("Cronbach's alpha para severidad: {:.3f} ({})".format(alpha_sev, categorize_alpha(alpha_sev)))

# Exportar los resultados al archivo de estadísticas como csv
alpha_results = pd.DataFrame({
    'Métrica': ['Frecuencia', 'Intensidad', 'Severidad'],
    'Cronbach_Alpha': [alpha_freq, alpha_int, alpha_sev],
    'Categoría': [categorize_alpha(alpha_freq), categorize_alpha(alpha_int), categorize_alpha(alpha_sev)]
})
alpha_results.to_csv(stats_dir / 'cronbach_alpha_results.csv', index=False)

Cronbach's alpha para frecuencia: 0.843 (muy buena)
Cronbach's alpha para intensidad: 0.985 (excelente, pero puede indicar redundancia)
Cronbach's alpha para severidad: 0.726 (buena)


## Test de medias

In [22]:
# crea una copia del df_final pero convierte las categorías a sus valores numéricos
df_numeric = df_final.copy()
for col in df_numeric.select_dtypes(include=['category', 'object']).columns:
    if isinstance(df_numeric[col].dtype, pd.CategoricalDtype):
        df_numeric[col] = df_numeric[col].cat.codes.replace(-1, np.nan)  # -1 es para NaN en categorías
    else:
        try:
            df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
        except Exception:
            continue

# exportamos a csv 
df_numeric.to_csv("../data/2_processed/df_final_numeric.csv", index=False)

In [23]:
# Realiza un test de medias entre los grupos con y sin SVI 
# para todas las columnas de df_numeric excepto el target y el score (puntaje_sindrome_visual_informatico)
from scipy.stats import mannwhitneyu, ttest_ind, chi2_contingency, fisher_exact

target = 'svi'
score_col = 'puntaje_sindrome_visual_informatico'
test_columns = [col for col in df_numeric.columns if col not in [target, score_col]]

results_mannwhitney = []
results_ttest = []
for col in test_columns:
    # Solo se prueban columnas numéricas
    if not pd.api.types.is_numeric_dtype(df_numeric[col]):
        continue
    group_svi = df_numeric[df_numeric[target] == 1][col].dropna()
    group_no_svi = df_numeric[df_numeric[target] == 0][col].dropna()
    
    # Mann–Whitney test
    if len(group_svi) > 0 and len(group_no_svi) > 0:
        try:
            u_stat, p_value = mannwhitneyu(group_svi, group_no_svi, alternative='two-sided')
        except Exception:
            u_stat, p_value = np.nan, np.nan
    else:
        u_stat, p_value = np.nan, np.nan

    results_mannwhitney.append({
        'metric': col,
        'mean_svi': group_svi.mean(),
        'mean_no_svi': group_no_svi.mean(),
        'u_stat': u_stat,
        'p_value': p_value,
        'differences_significativas': p_value < 0.01 if not np.isnan(p_value) else False
    })
    
    # t-test
    if len(group_svi) > 0 and len(group_no_svi) > 0:
        try:
            t_stat, p_val = ttest_ind(group_svi, group_no_svi, nan_policy='omit')
        except Exception:
            t_stat, p_val = np.nan, np.nan
    else:
        t_stat, p_val = np.nan, np.nan

    results_ttest.append({
        'metric': col,
        'mean_svi': group_svi.mean(),
        'mean_no_svi': group_no_svi.mean(),
        't_stat': t_stat,
        'p_value': p_val,
        'differences_significativas': p_val < 0.05 if not np.isnan(p_val) else False
    })

df_mannwhitney = pd.DataFrame(results_mannwhitney)
df_mannwhitney.to_csv(stats_dir / 'mannwhitney_svi_vs_no_svi.csv', index=False)

df_ttest = pd.DataFrame(results_ttest)
df_ttest.to_csv(stats_dir / 'ttest_svi_vs_no_svi.csv', index=False)

# ---------------------------
# Test de independencia para variables categóricas: Chi-cuadrado o Fisher
cat_test_columns = [col for col in df_final.columns 
                    if ((df_final[col].dtype == 'object') or (str(df_final[col].dtype).startswith('category')))
                    and col not in [target, score_col]]

results_chi = []
for col in cat_test_columns:
    # Construir la tabla de contingencia
    table = pd.crosstab(df_final[col], df_final[target])
    if table.empty:
        continue
    
    # Si la tabla es 2x2 usamos el test exacto de Fisher; si no, usamos Chi-cuadrado
    if table.shape == (2,2):
        try:
            _, p = fisher_exact(table)
            test_name = "Fisher"
        except Exception:
            p = np.nan
            test_name = "Fisher"
    else:
        try:
            chi2, p, dof, expected = chi2_contingency(table)
            test_name = "Chi-cuadrado"
        except Exception:
            p = np.nan
            test_name = "Chi-cuadrado"
    
    results_chi.append({
        'variable': col,
        'test': test_name,
        'p_value': p,
        'differences_significativas': p < 0.05 if not np.isnan(p) else False
    })

df_chi = pd.DataFrame(results_chi)
df_chi.to_csv(stats_dir / 'chi_square_fisher_svi_vs_no_svi.csv', index=False)

d:\encuesta_fatiga_visual\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
d:\encuesta_fatiga_visual\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
d:\encuesta_fatiga_visual\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
d:\encuesta_fatiga_visual\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: RuntimeWar

# Causalidad

In [59]:
# eliminamos las columnas que tiene los sufijos "_frecuencia", "_intensidad", "_severidad"
df_models = df_numeric.drop(columns=[col for col in df_numeric.columns if col.endswith(("_frecuencia", "_intensidad", "_severidad"))])

# eliminamos las columnas 'puntaje_sindrome_visual_informatico' y 'severity_check'
df_models = df_models.drop(columns=['puntaje_sindrome_visual_informatico', 'severity_check', 'trabajador'])

In [60]:
# hacemos un correlograma de sperman, anova y Kendall’s de todas las variables numéricas en df_models
plt.figure(figsize=(12,10))
sns.heatmap(df_models.corr(method='spearman'), annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5)
plt.title("Correlograma Spearman de Variables Numéricas")
plt.savefig(str(output_dir / "correlograma_variables_numericas_spearman.png"), bbox_inches='tight')
plt.close()

plt.figure(figsize=(12,10))
sns.heatmap(df_models.corr(method='kendall'), annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5)
plt.title("Correlograma Kendall de Variables Numéricas")
plt.savefig(str(output_dir / "correlograma_variables_numericas_kendall.png"), bbox_inches='tight')
plt.close()

In [61]:
# eliminamos las columnas que tengan correlacion mayor a 0.6 y menor a 0.6
corr_matrix = df_models.corr()
high_corr_var = np.where((corr_matrix > 0.6) | (corr_matrix < -0.6))
high_corr_var = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_var) if x != y and x < y]
high_corr_var

[('edad', 'estado_civil'),
 ('edad', 'experiencia_radiologia'),
 ('edad', 'ingresos_mensuales'),
 ('experiencia_radiologia', 'ingresos_mensuales'),
 ('experiencia_radiologia', 'lentes'),
 ('condiciones_oculares', 'lentes')]

In [62]:
df_models = df_models.drop(columns=['edad', 'experiencia_radiologia', 'condiciones_oculares', 'duracion_de_jornada'])

In [63]:
# exportar df_models a csv
df_models.to_csv("../data/2_processed/df_models.csv", index=False)

In [4]:
df_models = pd.read_csv("../data/2_processed/df_models.csv")

## Regresión lineal

In [12]:
y = df_models['svi']
X = df_models[['tiempo_de_exposicion', 'frecuencia_de_pausas', 'iluminacion', 'distancia_hacia_el_monitor']]
X = sm.add_constant(X)  # Agregar constante para el intercepto
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    svi   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.9053
Date:                Wed, 01 Oct 2025   Prob (F-statistic):              0.476
Time:                        22:10:20   Log-Likelihood:                -19.132
No. Observations:                  30   AIC:                             48.26
Df Residuals:                      25   BIC:                             55.27
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [65]:
# haz un modelo de regresión lineal que tenga como variable dependiente 'svi' y como variables independientes todas las demás variables numéricas en df_models
import statsmodels.api as sm
from statsmodels.formula.api import ols

y = df_models['svi']
X = df_models.drop(columns=['svi'])
X = sm.add_constant(X)  # Agregar constante para el intercepto
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    svi   R-squared:                       0.164
Model:                            OLS   Adj. R-squared:                 -0.212
Method:                 Least Squares   F-statistic:                    0.4364
Date:                Tue, 30 Sep 2025   Prob (F-statistic):              0.899
Time:                        22:40:01   Log-Likelihood:                -18.472
No. Observations:                  30   AIC:                             56.94
Df Residuals:                      20   BIC:                             70.96
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

## Regresión logística